In [5]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [6]:
mt5.initialize(
            path=os.getenv("MT5_PATH"),
            login=int(os.getenv("MT5_LOGIN")),
            password=os.getenv("MT5_PASSWORD"),
            server=os.getenv("MT5_SERVER"),
            timeout=int(os.getenv("MT5_TIMEOUT")),
            portable=eval(os.getenv("MT5_PORTABLE")),
        )

True

In [7]:
bars = pd.DataFrame(mt5.copy_rates_from_pos(
    "EURUSD",
    mt5.TIMEFRAME_H1,
    1,
    50,
))

bars['time'] = pd.to_datetime(bars['time'], unit='s')
bars.set_index('time', inplace=True)
bars.rename(columns={
    'tick_volume': 'tickvol',
    'real_volume': 'vol'
}, inplace=True)
bars = bars[['open', 'high', 'low', 'close', 'tickvol', 'vol', 'spread']]
bars.head()

,open,high,low,close,tickvol,vol,spread
time,,,,,,,
2025-05-19 17:00:00,1.12515,1.12578,1.12417,1.12538,4429,0,0
2025-05-19 18:00:00,1.12538,1.12599,1.12335,1.12363,3796,0,0
2025-05-19 19:00:00,1.12363,1.12397,1.12295,1.12353,2635,0,0
2025-05-19 20:00:00,1.12353,1.12388,1.12243,1.12303,2194,0,0
2025-05-19 21:00:00,1.12303,1.12370,1.12247,1.12351,1761,0,0


In [ ]:
def comput_rsi(self, prices: pd.Series) -> float:

    period = 14
    
    deltas = np.diff(prices)
    gains = np.where(deltas > 0, deltas, 0)
    losses = np.where(deltas < 0, -deltas, 0)

    average_gain = np.mean(gains[-period:])
    average_loss = np.mean(losses[-period:])

In [8]:
prices = bars['close']
np.diff(prices)

array([-1.75e-03, -1.00e-04, -5.00e-04,  4.80e-04,  5.10e-04, -2.00e-05,
        2.00e-04, -6.70e-04, -2.90e-04, -4.40e-04,  1.04e-03, -6.10e-04,
        1.28e-03, -2.70e-04,  1.23e-03,  3.00e-04,  1.87e-03, -1.79e-03,
       -3.10e-04, -1.08e-03,  1.30e-04, -1.15e-03,  1.15e-03,  2.27e-03,
       -1.54e-03,  3.60e-04,  1.72e-03, -1.50e-04,  8.00e-04,  3.90e-04,
       -2.70e-04, -7.00e-05,  7.40e-04,  7.20e-04,  2.90e-04,  2.35e-03,
        1.22e-03, -5.10e-04,  2.90e-04,  9.00e-05, -9.50e-04, -8.00e-05,
        1.23e-03,  4.00e-04, -1.54e-03,  1.11e-03,  2.30e-04,  4.00e-04,
        1.80e-04])

In [27]:
type(bars.iloc[-1])

pandas.core.series.Series

In [28]:
tick = mt5.symbol_info_tick("EURUSD")._asdict()
tick

{'time': 1745605393,
 'bid': 1.13628,
 'ask': 1.1363,
 'last': 0.0,
 'volume': 0,
 'time_msc': 1745605393217,
 'flags': 4,
 'volume_real': 0.0}

In [29]:
def get_latest_tick(symbol: str) -> dict:
    """Get the latest tick for a given symbol.
    Args:
        symbol (str): The symbol to fetch data for.
    Returns:
        dict: The latest tick data.
    Raises:
        ValueError: If the symbol is invalid.
        RuntimeError: If there is an error fetching data from MT5.
    """
    try:
        tick = mt5.symbol_info_tick(symbol)
        if tick is None:
            raise ValueError(f"No tick data received for symbol: {symbol}.")

    except Exception as e:
        raise RuntimeError(
            f"Error fetching tick data for symbol: {symbol}. "
            f"MT5 Error: {mt5.last_error()} "
            f"Exception: {str(e)}"
        )

    else:
        return tick._asdict()  # Convert the tick object to a dictionary

In [30]:
get_latest_tick("EURUSD")

{'time': 1745605393,
 'bid': 1.13628,
 'ask': 1.1363,
 'last': 0.0,
 'volume': 0,
 'time_msc': 1745605393217,
 'flags': 4,
 'volume_real': 0.0}

In [31]:
mt5.symbol_info("EURUSD").volume_min

0.01

In [32]:
mt5.symbol_info("EURUSD")._asdict()

{'custom': False,
 'chart_mode': 0,
 'select': True,
 'visible': True,
 'session_deals': 0,
 'session_buy_orders': 0,
 'session_sell_orders': 0,
 'volume': 0,
 'volumehigh': 0,
 'volumelow': 0,
 'time': 1745605393,
 'digits': 5,
 'spread': 2,
 'spread_float': True,
 'ticks_bookdepth': 10,
 'trade_calc_mode': 0,
 'trade_mode': 4,
 'start_time': 0,
 'expiration_time': 0,
 'trade_stops_level': 0,
 'trade_freeze_level': 0,
 'trade_exemode': 2,
 'swap_mode': 1,
 'swap_rollover3days': 3,
 'margin_hedged_use_leg': False,
 'expiration_mode': 15,
 'filling_mode': 2,
 'order_mode': 127,
 'order_gtc_mode': 0,
 'option_mode': 0,
 'option_right': 0,
 'bid': 1.13628,
 'bidhigh': 1.13914,
 'bidlow': 1.1316,
 'ask': 1.1363,
 'askhigh': 1.13975,
 'asklow': 1.1316,
 'last': 0.0,
 'lasthigh': 0.0,
 'lastlow': 0.0,
 'volume_real': 0.0,
 'volumehigh_real': 0.0,
 'volumelow_real': 0.0,
 'option_strike': 0.0,
 'point': 1e-05,
 'trade_tick_value': 1.0,
 'trade_tick_value_profit': 1.0,
 'trade_tick_value_loss'

In [33]:
mt5.account_info()._asdict()

{'login': 52247953,
 'trade_mode': 0,
 'leverage': 50,
 'limit_orders': 200,
 'margin_so_mode': 0,
 'trade_allowed': True,
 'trade_expert': True,
 'margin_mode': 2,
 'currency_digits': 2,
 'fifo_close': False,
 'balance': 10000.0,
 'credit': 0.0,
 'profit': 0.0,
 'equity': 10000.0,
 'margin': 0.0,
 'margin_free': 10000.0,
 'margin_level': 0.0,
 'margin_so_call': 100.0,
 'margin_so_so': 0.0,
 'margin_initial': 0.0,
 'margin_maintenance': 0.0,
 'assets': 0.0,
 'liabilities': 0.0,
 'commission_blocked': 0.0,
 'name': 'Ronald Javier Vezga Rueda',
 'server': 'ICMarketsSC-Demo',
 'currency': 'USD',
 'company': 'Raw Trading Ltd'}

In [34]:
mt5.symbol_info("EURUSD")._asdict()

{'custom': False,
 'chart_mode': 0,
 'select': True,
 'visible': True,
 'session_deals': 0,
 'session_buy_orders': 0,
 'session_sell_orders': 0,
 'volume': 0,
 'volumehigh': 0,
 'volumelow': 0,
 'time': 1745605393,
 'digits': 5,
 'spread': 2,
 'spread_float': True,
 'ticks_bookdepth': 10,
 'trade_calc_mode': 0,
 'trade_mode': 4,
 'start_time': 0,
 'expiration_time': 0,
 'trade_stops_level': 0,
 'trade_freeze_level': 0,
 'trade_exemode': 2,
 'swap_mode': 1,
 'swap_rollover3days': 3,
 'margin_hedged_use_leg': False,
 'expiration_mode': 15,
 'filling_mode': 2,
 'order_mode': 127,
 'order_gtc_mode': 0,
 'option_mode': 0,
 'option_right': 0,
 'bid': 1.13628,
 'bidhigh': 1.13914,
 'bidlow': 1.1316,
 'ask': 1.1363,
 'askhigh': 1.13975,
 'asklow': 1.1316,
 'last': 0.0,
 'lasthigh': 0.0,
 'lastlow': 0.0,
 'volume_real': 0.0,
 'volumehigh_real': 0.0,
 'volumelow_real': 0.0,
 'option_strike': 0.0,
 'point': 1e-05,
 'trade_tick_value': 1.0,
 'trade_tick_value_profit': 1.0,
 'trade_tick_value_loss'

In [35]:
mt5.positions_get()

()

## Research metodo Order_Send()

In [45]:
market_order_request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": "EURUSD",
    "volume": 0.10,
    "sl": 0.0,
    "tp": 0.0,
    "type": mt5.ORDER_TYPE_BUY,
    "deviation": 0,
    "magic": 1258,
    "comment": "FWK Market Order",
    "type_filling": mt5.ORDER_FILLING_IOC,
    "price": mt5.symbol_info("EURUSD").bid
    }

In [46]:
result = mt5.order_send(market_order_request)

In [47]:
result._asdict()

{'retcode': 10009,
 'deal': 826520625,
 'order': 1024179120,
 'volume': 0.1,
 'price': 1.1373,
 'bid': 1.1373,
 'ask': 1.1373,
 'comment': 'Request executed',
 'request_id': 1710381260,
 'retcode_external': 0,
 'request': TradeRequest(action=1, magic=1258, order=0, symbol='EURUSD', volume=0.1, price=1.13725, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=0, type_filling=1, type_time=0, expiration=0, comment='FWK Market Order', position=0, position_by=0)}

In [59]:
pending_order_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": "EURUSD",
        "volume": 0.10,
        "sl": 0.0,  
        "tp": 0.0,
        "type": mt5.ORDER_TYPE_BUY_LIMIT,
        "deviation": 0,
        "magic": 1258,
        "comment": "FWK Pending Order",
        "type_filling": mt5.ORDER_FILLING_IOC,
        "price": 1.136, 
        "type_time": mt5.ORDER_TIME_GTC,
    }

In [60]:
pending_result = mt5.order_send(pending_order_request)

In [61]:
pending_result._asdict()

{'retcode': 10009,
 'deal': 0,
 'order': 1024241328,
 'volume': 0.1,
 'price': 0.0,
 'bid': 0.0,
 'ask': 0.0,
 'comment': 'Request executed',
 'request_id': 1710381262,
 'retcode_external': 0,
 'request': TradeRequest(action=5, magic=1258, order=0, symbol='EURUSD', volume=0.1, price=1.136, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=2, type_filling=1, type_time=0, expiration=0, comment='FWK Pending Order', position=0, position_by=0)}

In [68]:
pending_order = mt5.orders_get(ticket=pending_result.order)[0]

In [69]:
pending_order

TradeOrder(ticket=1024241328, time_setup=1745607820, time_setup_msc=1745607820495, time_done=0, time_done_msc=0, time_expiration=0, type=2, type_time=0, type_filling=2, state=1, magic=1258, position_id=0, position_by_id=0, reason=3, volume_initial=0.1, volume_current=0.1, price_open=1.1360000000000001, sl=0.0, tp=0.0, price_current=1.13818, price_stoplimit=0.0, symbol='EURUSD', comment='FWK Pending Orde', external_id='')

In [70]:
cancel_request = {
    "action": mt5.TRADE_ACTION_REMOVE,
    "order": pending_order.ticket,
    "symbol": pending_order.symbol,
    "comment": "FWK Cancel Pending Order",
}

In [71]:
cancel_result = mt5.order_send(cancel_request)

In [72]:
cancel_result._asdict()

{'retcode': 10009,
 'deal': 0,
 'order': 1024241328,
 'volume': 0.0,
 'price': 0.0,
 'bid': 0.0,
 'ask': 0.0,
 'comment': 'Request executed',
 'request_id': 1710381263,
 'retcode_external': 0,
 'request': TradeRequest(action=8, magic=0, order=1024241328, symbol='EURUSD', volume=0.0, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=0, type_filling=0, type_time=0, expiration=0, comment='FWK Cancel Pending Order', position=0, position_by=0)}

In [51]:
position = mt5.positions_get(ticket=1024179120)[0]

In [54]:
position._asdict()

{'ticket': 1024179120,
 'time': 1745606434,
 'time_msc': 1745606434796,
 'time_update': 1745606434,
 'time_update_msc': 1745606434796,
 'type': 0,
 'magic': 1258,
 'identifier': 1024179120,
 'reason': 3,
 'volume': 0.1,
 'price_open': 1.1373,
 'sl': 0.0,
 'tp': 0.0,
 'price_current': 1.13719,
 'swap': 0.0,
 'profit': -1.1,
 'symbol': 'EURUSD',
 'comment': 'FWK Market Order',
 'external_id': ''}

In [53]:
mt5.positions_get()

(TradePosition(ticket=1024179120, time=1745606434, time_msc=1745606434796, time_update=1745606434, time_update_msc=1745606434796, type=0, magic=1258, identifier=1024179120, reason=3, volume=0.1, price_open=1.1373, sl=0.0, tp=0.0, price_current=1.13725, swap=0.0, profit=-0.5, symbol='EURUSD', comment='FWK Market Order', external_id=''),)

In [55]:
close_request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "position": position.ticket,
            "symbol": position.symbol,
            "volume": position.volume,
            "type": mt5.ORDER_TYPE_BUY if position.type == mt5.ORDER_TYPE_SELL else mt5.ORDER_TYPE_SELL,
            "comment": "FWK Close Position",
            "type_filling": mt5.ORDER_FILLING_IOC
        }

In [56]:
close_request

{'action': 1,
 'position': 1024179120,
 'symbol': 'EURUSD',
 'volume': 0.1,
 'type': 1,
 'comment': 'FWK Close Position',
 'type_filling': 1}

In [57]:
result = mt5.order_send(close_request)

In [58]:
result._asdict()

{'retcode': 10009,
 'deal': 826545022,
 'order': 1024204316,
 'volume': 0.1,
 'price': 1.13803,
 'bid': 1.13803,
 'ask': 1.13805,
 'comment': 'Request executed',
 'request_id': 1710381261,
 'retcode_external': 0,
 'request': TradeRequest(action=1, magic=0, order=0, symbol='EURUSD', volume=0.1, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=1, type_filling=1, type_time=0, expiration=0, comment='FWK Close Position', position=1024179120, position_by=0)}